In [1]:
import uproot
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
plt.rcParams['figure.figsize'] = (16,9)
plt.rcParams['font.size'] = 16

# Data preparation

In [3]:
year = '19'

In [4]:
files = uproot.pandas.iterate('../outputs/'+year+'/*.root', 't')#, branches=['label', 'be', 'm', 't', 'proc'])
raw = pd.concat([i for i in files])

In [5]:
df = raw.set_index('label')
df.shape

(130672, 15)

In [6]:
df.head()

,be,m,m_rec,p,align,t,proc,dedx[0],dedx[1],theta,len,chi,eks,pkl,phen
label,,,,,,,,,,,,,,,
537.5,537.454102,-1.0,712.913246,-1.0,-1.0,2,1,2103.525879,3704.649902,-1.0,-1.0,100000.0,730.251138,158.181629,22.973795
537.5,537.454102,-1.0,1002.441872,-1.0,-1.0,2,1,2429.589355,2343.751221,-1.0,-1.0,100000.0,1053.350155,323.507098,83.270065
537.5,537.454102,-1.0,746.018114,-1.0,-1.0,2,1,1540.333618,1844.079102,-1.0,-1.0,100000.0,773.137400,202.973921,22.707390
537.5,537.454102,-1.0,836.229260,-1.0,-1.0,2,1,2830.133301,2732.808838,-1.0,-1.0,100000.0,887.938840,298.590036,21.557570
537.5,537.454102,-1.0,999.865354,-1.0,-1.0,2,1,1657.372192,1684.059082,-1.0,-1.0,100000.0,1039.156206,283.045745,32.621197


Get Luminosity

In [7]:
lum = pd.read_csv('../inputs/'+year+'/lum', names=['label', 'lum'], index_col=['label'])

In [8]:
lum.head()

,lum
label,
550.1,499.5
550.0,1427.2
537.5,1221.9
562.5,1049.2
575.0,1058.9


Join luminosity to data frame

In [9]:
df = df.join(lum, how='inner')
df.shape

(127766, 16)

Remove kinfit data from searches

In [10]:
df = df.query('proc!=1')
df.drop(['proc', 'm_rec'], axis=1, inplace=True)
df.shape

(10069, 14)

Invariant M for fitting

In [11]:
invM = pd.DataFrame( df[['m']].groupby('label').apply(np.array).apply(np.ravel) )

In [12]:
invM.columns = ['mass']

In [13]:
invM['Nsig'] = None
invM['errN'] = None
invM['massSize'] = invM['mass'].apply(np.size)

In [14]:
invM.sort_values('massSize').head()

,mass,Nsig,errN,massSize
label,,,,
975.0,"[431.72125244140625, 496.7955322265625, 500.01...",None,None,8
912.5,"[493.04913330078125, 463.2291564941406, 545.03...",None,None,21
925.0,"[567.11767578125, 545.0470581054688, 499.53482...",None,None,23
887.5,"[508.60272216796875, 511.58795166015625, 501.0...",None,None,24
900.0,"[494.17205810546875, 574.067138671875, 540.238...",None,None,25


# Fitter

In [15]:
iterator = iter(invM.iterrows())

In [16]:
label, row = next(iterator)
print('Label |', label)

Label | 537.5


In [1]:
row.mass

NameError: name 'row' is not defined

In [ ]:
import zfit

In [5]:
mg

NameError: name 'mg' is not defined

In [122]:
lk

In [113]:
from skstats.hypotests.calculators import AsymptoticCalculator
from skstats.hypotests import Discovery
from skstats.hypotests.parameters import POI

In [124]:
calculator = AsymptoticCalculator(1, Minuit())

TypeError: __init__() takes at least 1 positional argument (0 given)

In [117]:
?AsymptoticCalculator